In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")
from tqdm.notebook import tqdm
import time
from IPython.display import clear_output

import ee, geemap

In [6]:
%load_ext autoreload
%autoreload 2
import sys  
sys.path.insert(0, '../../lib/')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
%reload_ext autoreload
import sand_classification as sc
import quality_mosaic as qm

# Part I: Get the locations from Google Drive

In [8]:
sheet_id = '13nF_pJ02Bd70cDJamuKbvZIkIdJ-kOI4O3Cx9K7Wzos'
# sheet_id = '1XqOtPkiE_Q0dfGSoyxrH730RkwrTczcRbDeJJpqRByQ'
sheet_name = 'mozambique'
# sheet_name = 'sample_1'
drive_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
drive_url

'https://docs.google.com/spreadsheets/d/13nF_pJ02Bd70cDJamuKbvZIkIdJ-kOI4O3Cx9K7Wzos/gviz/tq?tqx=out:csv&sheet=mozambique'

In [9]:
#Read in a google sheets with the ground truth data from string drive_loc
df = pd.read_csv(drive_url)
df

,ID,Latitude,Longitude,Class,Date,Site,Source,Country,Notes
0,moz-0,-25.571043,32.252410,sand,2021-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 10747CM applied for
1,moz-1,-24.505472,33.012067,sand,2022-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 10128CM Granted (2020-02-03 to 2030-02...
2,moz-2,-24.647689,33.283443,sand,2019-08-15,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 6528CM Granted (2013-08-22 to 2026-01-19)
3,moz-3,-24.633682,33.283166,sand,2019-08-15,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 8996CM Granted (2018-04-01 to 2028-04-01)
4,moz-4,-25.563075,32.209854,sand,2021-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 3507CM Granted (2015-03-31 to 2017-03-31)
5,moz-5,-25.353037,32.327312,sand,2016-12-23,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 8348C Applied (2016-09-61)


In [10]:
#look for any NaN values in the column 'date' or 'Longitude' or 'Latitude' in df_merged and drop them
df = df.dropna(subset=['Date', 'Longitude', 'Latitude']).reset_index(drop=True)
df

,ID,Latitude,Longitude,Class,Date,Site,Source,Country,Notes
0,moz-0,-25.571043,32.252410,sand,2021-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 10747CM applied for
1,moz-1,-24.505472,33.012067,sand,2022-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 10128CM Granted (2020-02-03 to 2030-02...
2,moz-2,-24.647689,33.283443,sand,2019-08-15,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 6528CM Granted (2013-08-22 to 2026-01-19)
3,moz-3,-24.633682,33.283166,sand,2019-08-15,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 8996CM Granted (2018-04-01 to 2028-04-01)
4,moz-4,-25.563075,32.209854,sand,2021-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 3507CM Granted (2015-03-31 to 2017-03-31)
5,moz-5,-25.353037,32.327312,sand,2016-12-23,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 8348C Applied (2016-09-61)


In [11]:
assert(len(df[df['Date'].isna() | df['Longitude'].isna() | df['Latitude'].isna()]) == 0)

# Part II: Select subset for training

Here we iterate over the df and iteratively pick and adjust the locations we want to keep

In [12]:
df.columns

Index(['ID', 'Latitude', 'Longitude', 'Class', 'Date', 'Site', 'Source',
       'Country', 'Notes'],
      dtype='object')

In [13]:
ee.Initialize()
# ee.Authenticate()

In [14]:
output = df.copy()
output

,ID,Latitude,Longitude,Class,Date,Site,Source,Country,Notes
0,moz-0,-25.571043,32.252410,sand,2021-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 10747CM applied for
1,moz-1,-24.505472,33.012067,sand,2022-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 10128CM Granted (2020-02-03 to 2030-02...
2,moz-2,-24.647689,33.283443,sand,2019-08-15,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 6528CM Granted (2013-08-22 to 2026-01-19)
3,moz-3,-24.633682,33.283166,sand,2019-08-15,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 8996CM Granted (2018-04-01 to 2028-04-01)
4,moz-4,-25.563075,32.209854,sand,2021-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 3507CM Granted (2015-03-31 to 2017-03-31)
5,moz-5,-25.353037,32.327312,sand,2016-12-23,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 8348C Applied (2016-09-61)


In [64]:
# output = qm.setup_output_bands(df.copy(),False)
# output


In [39]:
%reload_ext autoreload
import sand_classification as sc
import quality_mosaic as qm

In [95]:
INDEX = 1
sampling_buffer_m = 5
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## Manual Loop

Keep running the following 2 cells until you're done with the manual sampling.


In [ ]:
s1_s2, sample = qm.get_s1s2_data(output, Map, INDEX, display_smap=False, mosaic_method='qm', sampling_buffer_m=sampling_buffer_m, median_samples=5,roi_buffer_m=400, obia=True)

## Use this immediately after the map is refreshed to either select or discard the sample

INDEX will auto update on calling this function

In [98]:
output, INDEX = qm.get_training_sample(output, s1_s2, sample, Map, INDEX, display_clusters=False, obia=True)
output.iloc[INDEX-1]

Original marker accepted
Kept Observation


ID                                                              moz-1
Latitude                                                   -24.504898
Longitude                                                   33.011966
Class                                                            sand
Date                                                       2022-07-22
Site                                                          fluvial
Source                   https://portals.landfolio.com/mozambique/en/
Country                                                    Mozambique
Notes               License 10128CM Granted (2020-02-03 to 2030-02...
B2_mean                                                   1325.271362
B3_mean                                                   1971.426392
B4_mean                                                   2716.062012
B8_mean                                                   3296.224854
B8A_mean                                                  3324.782959
B11_mean            

## Final Output

In [99]:
output

,ID,Latitude,Longitude,Class,Date,Site,Source,Country,Notes,B2_mean,...,B8A_mean,B11_mean,B12_mean,VV_mean,VH_mean,mTGSI_mean,BSI_mean,NDWI_mean,keep,location_tweaked
0,moz-0,-25.567143,32.253844,sand,2021-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 10747CM applied for,1582.960815,...,3790.470703,5268.176270,4925.206055,-17.645203,-27.891870,0.170225,0.219581,-0.240858,True,True
1,moz-1,-24.504898,33.011966,sand,2022-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 10128CM Granted (2020-02-03 to 2030-02...,1325.271362,...,3324.782959,4737.139648,4739.217285,-16.774797,-27.074543,0.201724,0.234520,-0.251508,True,False
2,moz-2,-24.647689,33.283443,sand,2019-08-15,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 6528CM Granted (2013-08-22 to 2026-01-19),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
3,moz-3,-24.633682,33.283166,sand,2019-08-15,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 8996CM Granted (2018-04-01 to 2028-04-01),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
4,moz-4,-25.563075,32.209854,sand,2021-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 3507CM Granted (2015-03-31 to 2017-03-31),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
5,moz-5,-25.353037,32.327312,sand,2016-12-23,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 8348C Applied (2016-09-61),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False


In [119]:
#delete the last row of output
output = output.drop(output.index[INDEX])
output

,ID,Latitude,Longitude,Class,Date,Site,Source,Country,Notes,B2,...,B8A,B11,B12,VV,VH,mTGSI,BSI,NDWI,keep,location_tweaked
0,moz-0,-25.571043,32.252410,sand,2021-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 10747CM applied for,1424.775801,...,3415.000000,4678.000000,4285.000000,-15.051572,-26.791706,0.162989,0.218167,-0.223185,True,True
1,moz-1,-24.505472,33.012067,sand,2022-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 10128CM Granted (2020-02-03 to 2030-02...,1323.618705,...,3344.366906,4756.661871,4745.467626,-15.421135,-27.160017,0.204982,0.238582,-0.256787,True,False
2,moz-2,-24.647689,33.283443,sand,2019-08-15,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 6528CM Granted (2013-08-22 to 2026-01-19),1025.946237,...,3013.000000,4363.000000,4366.000000,-18.701325,-29.746097,0.219025,0.251916,-0.294739,True,False
3,moz-3,-24.633682,33.283166,sand,2019-08-15,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 8996CM Granted (2018-04-01 to 2028-04-01),986.097826,...,2851.134058,4177.449275,4213.413043,-18.801203,-33.041720,0.224025,0.254158,-0.304060,True,False
4,moz-4,-25.563075,32.209854,sand,2021-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 3507CM Granted (2015-03-31 to 2017-03-31),1508.221429,...,3616.185714,5096.050000,4711.221429,-14.737598,-25.256125,0.171833,0.225222,-0.233366,True,False
5,moz-5,-25.353037,32.327312,sand,2022-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 8348C Applied (2016-09-61),1336.747331,...,3605.000000,5036.000000,4777.000000,-16.226424,-24.205281,0.180526,0.225067,-0.293994,True,True


In [120]:
output.to_csv('../../data/mozambique-gt-final-locations.csv')
#save fm_df to a pickle file
output.to_pickle('../../data/mozambique-gt-final-locations.pkl')